In [20]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig


tokenizer = AutoTokenizer.from_pretrained("./new_tokenizer/")
model_path = "icebreaker/checkpoint-2200"
model = AutoModelForCausalLM.from_pretrained(model_path)


generation_config = GenerationConfig(
    num_return_sequences=1,
    max_length=80,
    top_k=50,
    top_p=0.95,
    do_sample=True,
    eos_token_id=model.config.eos_token_id,
)

model.save_pretrained("icebreaker-final/")
generation_config.save_pretrained("icebreaker-final/")
tokenizer.save_pretrained("icebreaker-final/")

model = AutoModelForCausalLM.from_pretrained("icebreaker-final/")
tokenizer = AutoTokenizer.from_pretrained("icebreaker-final/")
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=32000, bias=False)
)

In [21]:
# model size
print("model size: ", model.num_parameters())

model size:  110418432


In [22]:
from transformers import TextGenerationPipeline 


# Create a text generation pipeline
pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer)

generated_text = pipeline("<Fréttir> <Innlent>")

# Print the generated text
print(generated_text[0]["generated_text"] + "...")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<Fréttir> <Innlent> <Bæjar- og sveitarfélög> <Bæjar- og sveitarfélög> <Bæjar- og sveitarfélög> Verkefni og sveitarfélaga hafa gert ráð fyrir því að hægt verði að draga úr útblæstri gróðurhúsalofttegunda í sumar en gert er ráð fyrir að það komi til greina að hafa áhrif á losun jarðar. Þetta kemur fram í ályktun umhverfis- og nýsköpunarráðuneyti sem fram fór í morgun. Hann segir að áætlað...


In [19]:
inputs = tokenizer("<Fréttir>", return_tensors="pt")

# outputs = model.generate(**inputs, penalty_alpha=0.6, top_k=4, max_new_tokens=100)
# outputs = model.generate(**inputs, do_sample=True, num_beams=1, max_new_tokens=100)
# outputs = model.generate(**inputs, num_beams=5, do_sample=True, max_new_tokens=100)
outputs = model.generate(
    **inputs, num_beams=5, num_beam_groups=5, max_new_tokens=100, diversity_penalty=1.3
)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

ValueError: `diversity_penalty` is not 0.0 or `num_beam_groups` is not 1, triggering group beam search. In this generation mode, `do_sample` must be set to `False`

In [24]:
# # Save model
# from huggingface_hub import HfApi

# repo_name = "icebreaker"

# api = HfApi()
# model.push_to_hub(repo_name)
# tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sigurdur/icebreaker/commit/deb45b04150737ac295cb5d4987acae48a28a33e', commit_message='Upload tokenizer', commit_description='', oid='deb45b04150737ac295cb5d4987acae48a28a33e', pr_url=None, pr_revision=None, pr_num=None)